# BrainAge Project

In [7]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.model_selection import KFold
from sklearn.svm import SVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt
import datetime

In [8]:
# Parameters
# Config
toy = False
run_all = False

path = '/home/ubuntu/fmriprep/'
# output_dir = '/output/' #AWS cloud
output_dir = 'data/CNP_T1/' #For Daniel's computer

n_jobs = 20 #amount of cores
cv=4

description = 'CNP_T1_validation_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [9]:
# Load subject ids
if run_all:
    files = os.listdir(path)
    subj = [n[4:9] for n in files if n.startswith('sub')] #here we will filter the file names to leave only subj number as bellow

    input_dir = '/home/ubuntu/'
    file = 'subjlist_wfiles_CNP.txt'
    with open(input_dir+file, 'r') as f:
        lines = f.readlines()

    subj = [n.strip()[4:] for n in lines]

#     This is only done once
#     random_subj = subj[:]
#     random.shuffle(random_subj)

In [10]:
# Load ages
# df = pd.read_csv('participants.tsv', sep='\t')

# # take age from subj id
# y_age = []
# for i in random_subj:
#     a = df.loc[df['participant_id'] == 'sub-'+i] #whole row
#     y_age.append(int(a.age))


# # Load IQ
# df = pd.read_csv('add file here', sep='\t')

# # take age from subj id
# y_age = []
# for i in subj:
#     a = df.loc[df['participant_id'] == 'sub-'+i] #whole row
#     y_iq.append(int(a.id))


In [11]:
# # Load volumes for each subject into a dictionary
# d = {}
# for i in range(len(subj)):
#     img = nib.load(path+'sub-'+subj[i]+'_T1w_space_MNI152NLin2009cAsym_preproc.nii.gz').get_fdata()
#     d[subj[i]] = img

if run_all:
    filenames = []
    # data = []
    for i in range(len(random_subj)):
    #     img = nib.load(path+'sub-'+subj[i]+/anat/'sub-'+subj[i]+'_T1w_space_MNI152NLin2009cAsym_preproc.nii.gz').get_fdata()
    #     data.append(img)
        filenames.append(path+'sub-'+random_subj[i]+'/anat/sub-'+random_subj[i]+'_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz')

# data  = np.array(data) 

## Let's see the size of a single volume
# data.shape

# # Flatten 3D volume and append to a list (or feed 3D volume to nilearn)
# X = []

# for i in d.values():
#     flattened = i.flatten()
#     X.append(flattened)
    
# # Turn list into array 
# X = np.array(X)

In [12]:
# filenames

In [13]:
# Affine resample

# from nilearn.image import resample_to_img
# # img = data.affine

# resampled_imgs = []
# resampled_imgs.append(nib.load(path+filenames[0]))
                      
# for i in range(1,len(filenames)):
#     resampled_img = resample_to_img(path+filenames[i], path+filenames[0])
#     resampled_imgs.append(resampled_img)

# for i in resampled_imgs:
#     print(i.affine)

In [14]:
if run_all:
    path_and_filenames = [path+n for n in filenames]

    nifti_masker = NiftiMasker(
        standardize=False,
        smoothing_fwhm=2, mask_strategy='epi',
        memory='nilearn_cache')  # cache options
    gm_maps_masked = nifti_masker.fit_transform(filenames)
    n_samples, n_features = gm_maps_masked.shape
    print("%d samples, %d features" % (n_samples, n_features))

In [15]:
if run_all:
    print("%d samples, %d features" % (n_samples, n_features))

In [16]:
if run_all:
    %matplotlib inline
    from nilearn import plotting
    plotting.plot_roi(nifti_masker.mask_img_)
    plotting.plot_roi(nifti_masker.mask_img_, bg_img=filenames[0])
    plt.savefig(output_dir+'nilearn_mask')

In [17]:
if run_all:
    X_train = gm_maps_masked[:84]
    X_test = gm_maps_masked[84:]
    y_train = y_age[:84]
    y_test = y_age[84:]
    random_subj_train = random_subj[:84]
    random_subj_test = random_subj[84:]

    np.savez_compressed(output_dir+log_file+'train_set',a=X_train, b=y_train, c=random_subj_train)
    np.savez_compressed(output_dir+log_file+'test_set',a=X_test, b=y_test, c=random_subj_test)
else:
    loaded = np.load(output_dir+'train_set.npz')
    X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']


In [18]:
print(y_train.shape)
print(X_train.shape)

(84,)
(84, 1899247)


## Gridsearch 
### Models
* Linear Support vector regressor
    * C = 0.0001, 0.001, 0.1, 1.0, 10.0

### Feature selection
* ANOVA
    * 2000 features
    * 50 k
    * 100k features
    * 500k
    * Use all features (176*256*256=11534336)

In [22]:
from sklearn.pipeline import Pipeline

toy = True

if toy:
    X_train = X_train[:8]
    y_train = y_train[:8]


# Remove features with too low between-subject variance
# variance_threshold = 

# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# feature_selection = SelectKBest(f_regression)

# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
               

anova_svr = Pipeline([
    ('anova', SelectKBest(f_regression)),
    ('svr', SVR())])

parameters = [{'anova__k': [5000,100000, 'all'],
               'svr__C': [0.00001, 0.0001, 0.001, 0.01, 1, 10]}]

# parameters = [{'anova__k': [5000],
#                'svr__C': [0.00001, 0.0001]}]


grid = GridSearchCV(anova_svr, cv=cv, n_jobs=n_jobs, param_grid=parameters, scoring='neg_mean_absolute_error')
grid.fit(X_train, y_train)

# mean_scores = np.array(grid.cv_results_['neg_mean_absolute_error'])
# mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# # select score for best C
# mean_scores = mean_scores.max(axis=0)
# bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
#                (len(reducer_labels) + 1) + .5)






/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:194: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:194: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:194: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('anova', SelectKBest(k=10, score_func=<function f_regression at 0x1114fbf28>)), ('svr', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=20,
       param_grid=[{'anova__k': [5000], 'svr__C': [1e-05, 0.0001]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [23]:
print('best score: ',grid.best_score_)
print('best params: ',grid.best_params_)
print('\n\nfullresults: \n', grid.cv_results_)
df1 = pd.DataFrame(grid.cv_results_)
df1

best score:  -7.875
best params:  {'anova__k': 5000, 'svr__C': 1e-05}


fullresults: 
 {'mean_fit_time': array([5.27354044, 5.46070498]), 'std_fit_time': array([0.41653217, 0.55124428]), 'mean_score_time': array([1.33596385, 1.25342274]), 'std_score_time': array([0.03757027, 0.08407761]), 'param_anova__k': masked_array(data=[5000, 5000],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_svr__C': masked_array(data=[1e-05, 0.0001],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'anova__k': 5000, 'svr__C': 1e-05}, {'anova__k': 5000, 'svr__C': 0.0001}], 'split0_test_score': array([-7.5, -7.5]), 'split1_test_score': array([-9.5, -9.5]), 'split2_test_score': array([-8., -8.]), 'split3_test_score': array([-6.5, -6.5]), 'mean_test_score': array([-7.875, -7.875]), 'std_test_score': array([1.08253175, 1.08253175]), 'rank_test_score': array([1, 1], dtype=int32), 'split0_train_score': array([-7.66665667, -7.6

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_scor

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_anova__k,param_svr__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,5.273540,0.416532,1.335964,0.037570,5000,1e-05,"{'anova__k': 5000, 'svr__C': 1e-05}",-7.5,-9.5,-8.0,-6.5,-7.875,1.082532,1,-7.666657,-5.16666,-5.99999,-6.16666,-6.249992,0.901387
1,5.460705,0.551244,1.253423,0.084078,5000,0.0001,"{'anova__k': 5000, 'svr__C': 0.0001}",-7.5,-9.5,-8.0,-6.5,-7.875,1.082532,1,-7.666567,-5.16660,-5.99990,-6.16660,-6.249917,0.901377


In [24]:
age_pred = grid.predict(X_train)
mean_abs_error = np.mean(np.abs(age_pred-y_train))
df1 = pd.DataFrame(grid.cv_results_)

with open(output_dir+'log_'+log_file+'.txt', '+a') as f:
    f.write('best score: '+str(grid.best_score_))
    f.write('best params: '+str(grid.best_params_))
    f.write('\n\nfullresults: \n'+str(grid.cv_results_)+'\n')
    f.write(str(df1)+'\n')
    f.write(str(age_pred)+'\n')
    f.write(str(y_train)+'\n')
    f.write(str(mean_abs_error)+'\n')
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_scor